In [1]:
import os

os.chdir('/Users/nikhilisukapalli/Downloads/Capstone/mag_papers_0')
os.getcwd()

'/Users/nikhilisukapalli/Downloads/Capstone/mag_papers_0'

In [2]:
import networkx as nx

G = nx.read_gpickle("mag_graph_cleaned.gpickle")

In [3]:
print(f"Number of papers (nodes): {G.number_of_nodes()}")
print(f"Number of citations (edges): {G.number_of_edges()}")

Number of papers (nodes): 2393685
Number of citations (edges): 225071


In [4]:
page_ranks = nx.pagerank(G, alpha=0.85)
print(f"Number of pages: {len(page_ranks)}")

Number of pages: 2393685


In [8]:
import pickle

with open("pageranks.pkl", "wb") as f:
    pickle.dump(page_ranks, f)

In [5]:
import torch
print(torch.__version__)
print(torch.__file__)

import transformers
import sentence_transformers

print(transformers.__version__)
print(sentence_transformers.__version__)

2.2.2
/Users/nikhilisukapalli/opt/anaconda3/envs/graphqa2/lib/python3.9/site-packages/torch/__init__.py


/Users/nikhilisukapalli/opt/anaconda3/envs/graphqa2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.51.3
4.1.0


In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
embedding = model.encode("This is a test sentence")
print(embedding.shape)

(384,)


In [11]:
for node_id, data in G.nodes(data=True):
    print(data)
    break

{'title': 'Electron Spin Resonance Investigations of Oxygen-Centered Free Radicals in Biological Systems', 'year': 1988, 'authors': ['Ronald P. Mason', 'Kim M. Morehouse'], 'fos': ['Hydroxyl radical', 'Autoxidation', 'Oxygen', 'Hyperfine structure', 'Photochemistry', 'Nuclear magnetic resonance', 'Radical', 'Electron paramagnetic resonance', 'Superoxide', 'Xanthine oxidase', 'Chemistry'], 'abstract': 'Oxygen-centered free radicals have been detected directly with ESR in a variety of biological processes such as lipid autoxidation,1 the enzymatic formation of superoxide by xanthine oxidase,2 and hydroxyl radical formation by the Y-irradiation of ice.3 Since the common isotope of oxygen is spin-less, no nuclear hyperfine interaction is possible, and the g-value and any hydrogen hyperfine coupling provide the only criteria for distinguishing an oxygen-centered free radical from other free radicals. The greatest limitation on the direct detection of oxygen-centered free radicals, aside fro

In [ ]:
paper_ids = []
paper_texts = []

for node_id, data in G.nodes(data=True):
    if data.get('title') and data.get('abstract'):
        text = (data.get('title', '') + ' ' + data.get('abstract', '')).strip()
        if text:
            paper_ids.append(node_id)
            paper_texts.append(text)

print(f"Parsed {len(paper_texts)} papers")

Parsed 2393685 papers


In [8]:
import psutil

def print_memory_usage():
    process = psutil.Process(os.getpid())
    mem_mb = process.memory_info().rss / (1024 * 1024)
    print(f"Current RAM usage: {mem_mb:.2f} MB")

In [9]:
from tqdm import tqdm
import time
import numpy as np

In [ ]:
embeddings = []
batch_sizes = [64, 128, 256]
chunk_size = 10000
timings = {}

for i, size in enumerate(batch_sizes):
    start = i * chunk_size
    end = start + chunk_size
    sample = paper_texts[start:end]
    print(f"Processing batch size {size} on samples {start}–{end}...")

    start_time = time.time()
    batch_embeddings = model.encode(sample, show_progress_bar=True, batch_size=size)
    timings[size] = time.time() - start_time
    embeddings.extend(batch_embeddings)
    print(f"Batch size {size} took {timings[size]:.2f} seconds")
    print()

Processing batch size 64 on samples 0–10000...


Batches: 100%|██████████| 157/157 [09:33<00:00,  3.65s/it]


Batch size 64 took 574.10 seconds

Processing batch size 128 on samples 10000–20000...


Batches: 100%|██████████| 79/79 [11:05<00:00,  8.42s/it]


Batch size 128 took 666.29 seconds

Processing batch size 256 on samples 20000–30000...


Batches: 100%|██████████| 40/40 [09:48<00:00, 14.72s/it]

Batch size 256 took 589.64 seconds



In [26]:
embeddings2 = []
start = 30000
end = 500000
samples = paper_texts[start:end]
print(f"Processing batch size 256 on samples {start}–{end}...")

start_time = time.time()

for i in tqdm(range(0, len(samples), 256)):
    batch_embeddings2 = model.encode(samples[i:i + 256])
    embeddings2.extend(batch_embeddings2)

    if (i // 256) % 100 == 0:
        print(f"Completed {(i // 256)} batches...")
        print_memory_usage()
        print()

duration2 = time.time() - start_time
print(f"Batch size 256 took {duration2:.2f} seconds")

Processing batch size 256 on samples 30000–500000...


  0%|          | 1/1836 [00:08<4:35:08,  9.00s/it]

Completed 0 batches...
Current RAM usage: 1896.48 MB



  6%|▌         | 101/1836 [18:54<5:23:30, 11.19s/it]

Completed 100 batches...
Current RAM usage: 1943.68 MB



...

Completed 1700 batches...
Current RAM usage: 349.45 MB



 98%|█████████▊| 1801/1836 [5:52:30<05:52, 10.08s/it]

Completed 1800 batches...
Current RAM usage: 422.50 MB



100%|██████████| 1836/1836 [5:58:20<00:00, 11.71s/it]

Batch size 256 took 21500.97 seconds


In [29]:
embeddings3 = []
start = 500000
end = 1000000
samples = paper_texts[start:end]
print(f"Processing batch size 256 on samples {start}–{end}...")

start_time = time.time()

for i in tqdm(range(0, len(samples), 256)):
    batch_embeddings3 = model.encode(samples[i:i + 256])
    embeddings3.extend(batch_embeddings3)

    if (i // 256) % 100 == 0:
        print(f"Completed {(i // 256)} batches...")
        print_memory_usage()
        print()

duration3 = time.time() - start_time
print(f"Batch size 256 took {duration3:.2f} seconds")

Processing batch size 256 on samples 500000–1000000...


  0%|          | 1/1954 [00:09<5:07:17,  9.44s/it]

Completed 0 batches...
Current RAM usage: 925.82 MB



  5%|▌         | 101/1954 [22:54<6:42:35, 13.04s/it]

Completed 100 batches...
Current RAM usage: 421.03 MB



...

Completed 1800 batches...
Current RAM usage: 687.35 MB



 97%|█████████▋| 1901/1954 [8:37:19<12:31, 14.17s/it]

Completed 1900 batches...
Current RAM usage: 728.07 MB



100%|██████████| 1954/1954 [8:48:52<00:00, 16.24s/it]

Batch size 256 took 31732.62 seconds


In [ ]:
print(len(embeddings), len(embeddings2), len(embeddings3))

30000 470000 500000


In [ ]:
np.savez_compressed(
    'embeddings_1M.npz',
    embeddings=np.array(embeddings, dtype=object),
    embeddings2=np.array(embeddings2, dtype=object),
    embeddings3=np.array(embeddings3, dtype=object)
)

In [17]:
embeddings4 = []
start = 1000000
end = 1500000
samples = paper_texts[start:end]
print(f"Processing batch size 256 on samples {start}–{end}...")

start_time = time.time()

for i in tqdm(range(0, len(samples), 256)):
    batch_embeddings4 = model.encode(samples[i:i + 256])
    embeddings4.extend(batch_embeddings4)

    if (i // 256) % 100 == 0:
        print(f"Completed {(i // 256)} batches...")
        print_memory_usage()
        print()

duration4 = time.time() - start_time
print(f"Batch size 256 took {duration4:.2f} seconds")

Processing batch size 256 on samples 1000000–1500000...


  0%|          | 1/1954 [00:13<7:31:58, 13.89s/it]

Completed 0 batches...
Current RAM usage: 1189.71 MB



  5%|▌         | 101/1954 [20:59<6:25:08, 12.47s/it]

Completed 100 batches...
Current RAM usage: 2392.49 MB



...

Completed 1800 batches...
Current RAM usage: 498.56 MB



 97%|█████████▋| 1901/1954 [21:08:31<09:29, 10.75s/it]

Completed 1900 batches...
Current RAM usage: 514.81 MB



100%|██████████| 1954/1954 [21:17:29<00:00, 39.23s/it]

Batch size 256 took 76649.67 seconds


In [18]:
print(len(embeddings4))

np.savez_compressed(
    'embeddings_1.5M.npz',
    embeddings4=np.array(embeddings4, dtype=object)
)

500000


In [10]:
embeddings5 = []
start = 1500000
end = 2000000
samples = paper_texts[start:end]
print(f"Processing batch size 256 on samples {start}–{end}...")

start_time = time.time()

for i in tqdm(range(0, len(samples), 256)):
    batch_embeddings5 = model.encode(samples[i:i + 256])
    embeddings5.extend(batch_embeddings5)

    if (i // 256) % 100 == 0:
        print(f"Completed {(i // 256)} batches...")
        print_memory_usage()
        print()

duration5 = time.time() - start_time
print(f"Batch size 256 took {duration5:.2f} seconds")

Processing batch size 256 on samples 1500000–2000000...


  0%|          | 1/1954 [00:23<12:59:29, 23.95s/it]

Completed 0 batches...
Current RAM usage: 4416.01 MB



  5%|▌         | 101/1954 [23:24<8:57:03, 17.39s/it]

Completed 100 batches...
Current RAM usage: 264.92 MB



...

Completed 1800 batches...
Current RAM usage: 1083.92 MB



 97%|█████████▋| 1901/1954 [7:15:52<11:35, 13.12s/it]    

Completed 1900 batches...
Current RAM usage: 704.33 MB



100%|██████████| 1954/1954 [7:26:33<00:00, 13.71s/it]

Batch size 256 took 26793.09 seconds


In [11]:
print(len(embeddings5))

np.savez_compressed(
    'embeddings_2M.npz',
    embeddings5=np.array(embeddings5, dtype=object)
)

500000


In [13]:
embeddings6 = []
start = 2000000
samples = paper_texts[start:]
print(f"Processing batch size 256 on samples {start}–{len(paper_texts)}...")

start_time = time.time()

for i in tqdm(range(0, len(samples), 256)):
    batch_embeddings6 = model.encode(samples[i:i + 256])
    embeddings6.extend(batch_embeddings6)

    if (i // 256) % 100 == 0:
        print(f"Completed {(i // 256)} batches...")
        print_memory_usage()
        print()

duration6 = time.time() - start_time
print(f"Batch size 256 took {duration6:.2f} seconds")

Processing batch size 256 on samples 2000000–2393685...


  0%|          | 1/1538 [00:12<5:16:54, 12.37s/it]

Completed 0 batches...
Current RAM usage: 1385.68 MB



  7%|▋         | 101/1538 [19:23<4:32:25, 11.37s/it]

Completed 100 batches...
Current RAM usage: 1483.93 MB



...

Completed 1400 batches...
Current RAM usage: 773.73 MB



 98%|█████████▊| 1501/1538 [5:08:37<06:29, 10.52s/it]

Completed 1500 batches...
Current RAM usage: 482.22 MB



100%|██████████| 1538/1538 [5:16:09<00:00, 12.33s/it]

Batch size 256 took 18969.82 seconds


In [14]:
print(len(embeddings6))

np.savez_compressed(
    'embeddings_2.4M.npz',
    embeddings6=np.array(embeddings6, dtype=object)
)

393685
